## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BSBSBSBS'
meas='TOBS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1580
+------------------+-------------------+------------------+------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+--------------------+-----------+-----------+------+--------------------+------+
|           coeff_1|            coeff_2|           coeff_3|           coeff_4|elevation|   label|latitude|longitude|measurement|             res_1|             res_2|             res_3|             res_4|            res_mean|    station|  total_var|undefs|              vector|  year|
+------------------+-------------------+------------------+------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+--------------------+-----------+-----------+------+--------------------+------+
| -357.316410504599|-129.13504460482713| 75.15483323926412|116.8914068

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'coeff_4', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_4', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [27]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
# Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_1, coeff_2, coeff_3, coeff_4'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_1, coeff_2, coeff_3, coeff_4 FROM weather


station    year     coeff_1     coeff_2     coeff_3     coeff_4
0  USC00084095  2012.0 -357.316411 -129.135045   75.154833  116.891407
1  USC00085895  1964.0 -389.789783  132.145388  -44.991567   32.864147
2  USC00086478  1937.0 -454.895471   84.891480  211.594555 -215.931513
3  USC00084091  1943.0  349.014439  187.152924 -113.156737 -119.113056
4  USC00087600  1931.0 -328.720015  119.112398 -201.173454  -88.447765

In [28]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_4')
year_station_table.head(10)

station  USC00080488  USC00080611  USC00080737  USC00080739  USC00080992  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1907.0           NaN          NaN          NaN          NaN          NaN   
1908.0           NaN          NaN          NaN          NaN          NaN   
1909.0           NaN          NaN          NaN          NaN          NaN   
1910.0           NaN          NaN          NaN          NaN          NaN   

station  USC00081276  USC00081305  USC00081306  USC00081651  USC00081654  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1907.0           NaN          NaN          NaN          NaN          NaN   
1908.0           NaN          NaN          NaN          NaN          NaN   
1909.0           NaN          NaN          NaN          NaN          NaN   
1910.0           NaN          NaN          NaN          NaN          NaN   

station     ...       USC00087758  USC00087760  USC00088368  USC00088620  \
year        ...                                                            
1901.0      ...               NaN          NaN          NaN          NaN   
1902.0      ...               NaN          NaN          NaN          NaN   
1903.0      ...               NaN          NaN          NaN          NaN   
1904.0      ...               NaN          NaN          NaN          NaN   
1905.0      ...               NaN          NaN          NaN          NaN   
1906.0      ...               NaN          NaN          NaN          NaN   
1907.0      ...               NaN          NaN          NaN          NaN   
1908.0      ...               NaN          NaN          NaN          NaN   
1909.0      ...               NaN          NaN          NaN          NaN   
1910.0      ...               NaN          NaN          NaN          NaN   

station  USC00088780  USC00088841  USC00089219  USC00089511  USC00089520  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1907.0           NaN          NaN          NaN          NaN          NaN   
1908.0           NaN          NaN          NaN          NaN          NaN   
1909.0           NaN          NaN          NaN          NaN          NaN   
1910.0           NaN          NaN          NaN          NaN          NaN   

station  USW00092811  
year                  
1901.0           NaN  
1902.0           NaN  
1903.0           NaN  
1904.0           NaN  
1905.0           NaN  
1906.0           NaN  
1907.0           NaN  
1908.0           NaN  
1909.0           NaN  
1910.0           NaN  

[10 rows x 65 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [29]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station), ', fraction explained: ', 1-RMS(tbl_minus_station)/RMS(year_station_table)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year), ', fraction explained: ', 1-RMS(tbl_minus_year)/RMS(year_station_table)

total RMS                   =  144.22492196
RMS removing mean-by-station=  139.569721235 , fraction explained:  0.0322773669188
RMS removing mean-by-year   =  85.4703255484 , fraction explained:  0.407381717479


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 185.261958184
0 after removing mean by year    = 143.673181451
0 after removing mean by stations= 129.844628965
1 after removing mean by year    = 129.272391297
1 after removing mean by stations= 129.197365591
2 after removing mean by year    = 129.181393137
2 after removing mean by stations= 129.176985727
3 after removing mean by year    = 129.175493138
3 after removing mean by stations= 129.174896702
4 after removing mean by year    = 129.174627807
4 after removing mean by stations= 129.174496304
